In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
p1 = Path.cwd() / 'data'

In [3]:
fruit = pd.read_csv(p1 / 'state_fruit.csv', index_col=0)
fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


In [11]:
fruit.stack().rename_axis(['State', 'Fruit']).reset_index(name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [12]:
fruit2 = pd.read_csv(p1 / 'state_fruit2.csv')
fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [14]:
fruit2.set_index('State').stack().rename_axis(['State', 'Fruit']).reset_index(name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


In [17]:
fruit2.melt(id_vars='State', var_name='Fruit', value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


In [20]:
fruit2.melt(id_vars='State', value_vars=['Apple', 'Orange'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14


In [22]:
movie = pd.read_csv(p1 / 'movie.csv')
actor_cols = sorted([col for col in movie.columns if 'actor_' in col or 'movie_title' in col])
actor = movie[actor_cols]

In [23]:
actor.head()

,actor_1_facebook_likes,actor_1_name,actor_2_facebook_likes,actor_2_name,actor_3_facebook_likes,actor_3_name,movie_title
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,Avatar
1,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,Pirates of the Caribbean: At World's End
2,11000.0,Christoph Waltz,393.0,Rory Kinnear,161.0,Stephanie Sigman,Spectre
3,27000.0,Tom Hardy,23000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,The Dark Knight Rises
4,131.0,Doug Walker,12.0,Rob Walker,NaN,NaN,Star Wars: Episode VII - The Force Awakens


In [30]:
def ccn(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = (col_name[:5] + '_fb' + col_name[5:fb_idx-1])
    return col_name

In [31]:
actor2 = actor.rename(columns=ccn)
actor2.head()

,actor_fb_1,actor_1,actor_fb_2,actor_2,actor_fb_3,actor_3,movie_title
0,1000.0,CCH Pounder,936.0,Joel David Moore,855.0,Wes Studi,Avatar
1,40000.0,Johnny Depp,5000.0,Orlando Bloom,1000.0,Jack Davenport,Pirates of the Caribbean: At World's End
2,11000.0,Christoph Waltz,393.0,Rory Kinnear,161.0,Stephanie Sigman,Spectre
3,27000.0,Tom Hardy,23000.0,Christian Bale,23000.0,Joseph Gordon-Levitt,The Dark Knight Rises
4,131.0,Doug Walker,12.0,Rob Walker,NaN,NaN,Star Wars: Episode VII - The Force Awakens


In [32]:
stubs = ['actor', 'actor_fb']
actor2_tidy = pd.wide_to_long(actor2, stubnames=stubs, i=['movie_title'], j='actor_num', sep='_')
actor2_tidy.head()

,,actor,actor_fb
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0


In [36]:
pd.wide_to_long(actor2, stubnames=stubs, i=['movie_title'], j='actor_num', sep='_', suffix=r'\d+')

,,actor,actor_fb
movie_title,actor_num,,
Avatar,1,CCH Pounder,1000.0
Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
Spectre,1,Christoph Waltz,11000.0
The Dark Knight Rises,1,Tom Hardy,27000.0
Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0
...,...,...,...
Signed Sealed Delivered,3,Crystal Lowe,318.0
The Following,3,Sam Underwood,319.0
A Plague So Pleasant,3,David Chandler,0.0


In [37]:
df = pd.read_csv(p1 / 'stackme.csv')
df

,State,Country,a1,b2,Test,d,e
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [38]:
df2 = df.rename(columns={'a1':'group1_a1', 'b2':'group1_b2', 'd':'group2_a1', 'e':'group2_b2'})
df2

,State,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [39]:
pd.wide_to_long(df2, stubnames=['group1', 'group2'], i=['State', 'Country', 'Test'], j='Group', sep='_', suffix=r'\w+\d+')

group1  group2
State Country Test  Group                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

In [40]:
def usecol_func(name):
    return 'UGDS_' in name or name == 'INSTNM'

In [42]:
college = pd.read_csv(p1 / 'college.csv', usecols=usecol_func, index_col='INSTNM')
college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [43]:
col_stack = college.stack()
col_stack.head()

INSTNM                              
Alabama A & M University  UGDS_WHITE    0.0333
                          UGDS_BLACK    0.9353
                          UGDS_HISP     0.0055
                          UGDS_ASIAN    0.0019
                          UGDS_AIAN     0.0024
dtype: float64

In [44]:
col_stack.unstack()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137
...,...,...,...,...,...,...,...,...,...
Hollywood Institute of Beauty Careers-West Palm Beach,0.2182,0.4182,0.2364,0.0182,0.0000,0.0000,0.0000,0.0182,0.0909
Hollywood Institute of Beauty Careers-Casselberry,0.1200,0.3333,0.4400,0.0000,0.0000,0.0000,0.0400,0.0000,0.0667
Coachella Valley Beauty College-Beaumont,0.3284,0.1045,0.4925,0.0149,0.0299,0.0149,0.0149,0.0000,0.0000
